<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Funcs" data-toc-modified-id="Funcs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Funcs</a></span></li><li><span><a href="#Сбор-данных" data-toc-modified-id="Сбор-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Сбор данных</a></span></li><li><span><a href="#Оценка-MDE/мощности/traffic-size-average-метрик-на-примере-GMV-per-user-с-CUPED" data-toc-modified-id="Оценка-MDE/мощности/traffic-size-average-метрик-на-примере-GMV-per-user-с-CUPED-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Оценка MDE/мощности/traffic size average метрик на примере GMV per user с CUPED</a></span><ul class="toc-item"><li><span><a href="#Если-в-эксперименте-вы-будете-использовать-не-весь-траффик,-а-только-x%-от-всего,-то-нужно-выделить-подвыборку-соответствующего-размера" data-toc-modified-id="Если-в-эксперименте-вы-будете-использовать-не-весь-траффик,-а-только-x%-от-всего,-то-нужно-выделить-подвыборку-соответствующего-размера-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Если в эксперименте вы будете использовать не весь траффик, а только x% от всего, то нужно выделить подвыборку соответствующего размера</a></span></li><li><span><a href="#Применим-CUPED" data-toc-modified-id="Применим-CUPED-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Применим CUPED</a></span></li><li><span><a href="#Выделим-первую-группу" data-toc-modified-id="Выделим-первую-группу-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Выделим первую группу</a></span></li><li><span><a href="#sd-и-sample-estimate" data-toc-modified-id="sd-и-sample-estimate-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>sd и sample estimate</a></span></li><li><span><a href="#Оценим-MDE-на-данной-выборке" data-toc-modified-id="Оценим-MDE-на-данной-выборке-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Оценим MDE на данной выборке</a></span></li><li><span><a href="#Оценим-мощность-для-фиксированного-эффекта" data-toc-modified-id="Оценим-мощность-для-фиксированного-эффекта-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Оценим мощность для фиксированного эффекта</a></span></li><li><span><a href="#Оценим-необходимое-количество-наблюдений-для-фиксации-заданного-эффекта-на-заданной-alpha-с-заданной-мощностью" data-toc-modified-id="Оценим-необходимое-количество-наблюдений-для-фиксации-заданного-эффекта-на-заданной-alpha-с-заданной-мощностью-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Оценим необходимое количество наблюдений для фиксации заданного эффекта на заданной alpha с заданной мощностью</a></span></li></ul></li></ul></div>

## Imports

In [1]:
import os
import pathlib

import numpy as np
import pandas as pd
import sqlalchemy as sa
import statsmodels.api as sm
import statsmodels.stats.api as sms
from dotenv import dotenv_values
from scipy.stats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
os.environ["OPENBLAS_NUM_THREADS"] = "8"

In [3]:
config = dotenv_values("/home/jovyan/.env") 

def get_query_clickhouse(q: str) -> pd.DataFrame:
    """
    Function to import credentials and run query
    """
    ch_host = config['CH_HOST']
    ch_cert = config['CH_CERT']
    ch_port = config['CH_PORT']
    ch_db   = config['CH_READ_DB']
    ch_user = config['CH_READ_USER']
    ch_pass = config['CH_READ_PASS']
    
    engine = sa.create_engine(
        f"clickhouse+native://{ch_user}:"
        f"{ch_pass}@{ch_host}:"
        f"{ch_port}/{ch_db}?secure=True"
    )
    return pd.read_sql_query(q, con=engine)

In [4]:
metrics_data_table = "cdm.ab__metrics_data"
platform = 'ios'
alpha = 0.05 # alpha, для которой считаем MDE
power = 0.8 # мощность, для которой считаем MDE
test_fraction = 0.5 # Тест-контроль 50 на 50

## Funcs

In [5]:
def collect_data(start_date: datetime.date, end_date: datetime.date,
                 metrics_data_table: str, platform: str) -> pd.DataFrame:
    q = f"""
        select 
            anonymous_id, 
            orders,
            gmv_per_user
        from 
            {metrics_data_table}
        where 
        toDate(date_msk) between toDate('{start_date}') and toDate('{end_date}')  
        and platform='{platform}'
        """
    return get_query_clickhouse(q)

## Сбор данных

In [6]:
%%time
# соберем экспериментальные данные
exp_start_date = "2023-11-29"
exp_end_date = "2023-12-12"

exp_ios_data = collect_data(exp_start_date, exp_end_date, metrics_data_table, platform)

CPU times: user 13.9 s, sys: 1.96 s, total: 15.9 s
Wall time: 16.2 s


In [7]:
%%time
# соберем исторические данные (за ту же длительность)
obs_start_date = "2023-11-15"
obs_end_date = "2023-11-28"

obs_ios_data = collect_data(obs_start_date, obs_end_date, metrics_data_table, platform)

CPU times: user 10.7 s, sys: 3.05 s, total: 13.8 s
Wall time: 14.2 s


In [8]:
obs_ios_data = obs_ios_data.groupby("anonymous_id").sum().reset_index()
exp_ios_data = exp_ios_data.groupby("anonymous_id").sum().reset_index()

## Оценка MDE/мощности/traffic size average метрик на примере GMV per user с CUPED

In [9]:
exp_gmv_data = exp_ios_data.copy()
obs_gmv_data = obs_ios_data.rename({"gmv_per_user": "CUPED_X"}, axis=1)

### Если в эксперименте вы будете использовать не весь траффик, а только x% от всего, то нужно выделить подвыборку соответствующего размера

In [10]:
# для 50% трафика нужно раскомментить :)
# gmv_data = gmv_data.sample(frac=0.5)

### Применим CUPED

In [11]:
# объединим экспериментальные и исторические данные в один датасет
gmv_data = exp_gmv_data.merge(obs_gmv_data[["anonymous_id", "CUPED_X"]], how="left", on="anonymous_id")

In [12]:
# обработаем юзеров без истории
gmv_data["missing_CUPED"] = 0
gmv_data.loc[gmv_data["CUPED_X"].isna(), "missing_CUPED"] = 1
gmv_data = gmv_data.fillna(0)
print(f"Пользователей без истории {gmv_data.missing_CUPED.mean():.4%}")

Пользователей без истории 37.5723%


In [13]:
# в соответствие с процедурой CUPED'a случайным образом выделим тестовую и контрольную группы
gmv_data["treatment"] = np.random.choice([0,1], size=gmv_data.shape[0])

In [14]:
# применим CUPED, получив по итогу пост-cuped метрику CUPED_GMV
y_control = gmv_data.query("treatment==0")["gmv_per_user"]
X_cov_control = gmv_data.query("treatment==0")[["CUPED_X", "missing_CUPED"]]
y_hat = sm.OLS(y_control, X_cov_control).fit().predict(gmv_data[["CUPED_X", "missing_CUPED"]])
gmv_data["CUPED_GMV"] = gmv_data["gmv_per_user"] - y_hat

In [15]:
# зафиксируем снижение дисперсии
print(f"""
Исходная дисперсия = {gmv_data.gmv_per_user.std():.8}
Дисперсия после cuped = {gmv_data.CUPED_GMV.std():.8}
Уменьшение на {1-gmv_data.CUPED_GMV.std()/gmv_data.gmv_per_user.std():.3%}""")


Исходная дисперсия = 5342.2058
Дисперсия после cuped = 4056.955
Уменьшение на 24.058%


### Выделим первую группу
 assumption: тест и контроль делятся 50/50

In [16]:
gmv_test_observations = gmv_data.sample(frac=test_fraction)

### sd и sample estimate
Заметим, что est_gmv мы оцениваем по метрики без CUPED. Это делается так, поскольку est_gmv мы используем только для получения размера эффекта, а CUPED не изменяет размер эффекта.

In [17]:
sd_gmv = gmv_test_observations.CUPED_GMV.std() # считаем по метрике CUPED_GMV
est_gmv = gmv_test_observations.gmv_per_user.mean() # считаем по дефолтной метрике, потому что размер эффекта не изменится

In [18]:
sd_gmv, est_gmv

(4054.115726716741, 2036.2528089426496)

### Оценим MDE на данной выборке

In [19]:
ratio = (1-test_fraction) / (test_fraction) # пропорция контроль/тест
nobs_test = gmv_test_observations.shape[0] # количество наблюдений (юзеров) в тесте

In [20]:
effect_size_gmv = tt_ind_solve_power(power=power, nobs1=nobs_test, alpha=alpha, ratio=ratio)

In [21]:
# eff_size = mde_lift * est_gmv / sd_gmv

In [22]:
mde_gmv = effect_size_gmv * sd_gmv/est_gmv

In [23]:
# в процентах
print(f"MDE в % равно {mde_gmv:.8%}")

MDE в % равно 1.00006719%


In [24]:
# в абсолютных значениях
print(f"MDE в абсолютных значениях равно {mde_gmv*est_gmv:.8}")

MDE в абсолютных значениях равно 20.363896


### Оценим мощность для фиксированного эффекта

In [25]:
ratio = (1-test_fraction) / (test_fraction) # пропорция контроль/тест
lift_gmv = 0.0075 # размер эффекта в %, для которого оцениваем мощность
nobs_test = gmv_test_observations.shape[0] # количество наблюдений (юзеров) в тесте

In [26]:
eff_size_gmv = lift_gmv * est_gmv/sd_gmv

In [27]:
power_gmv = tt_ind_solve_power(effect_size=eff_size_gmv, nobs1=nobs_test, alpha=alpha, ratio=ratio)

In [28]:
# мощность для эффекта в 0.75% (lift_aov)
print(f"Мощность теста для эффекта в {lift_gmv:.4%} равна {power_gmv:.6}")

Мощность теста для эффекта в 0.7500% равна 0.556153


### Оценим необходимое количество наблюдений для фиксации заданного эффекта на заданной alpha с заданной мощностью

In [29]:
eff_size_gmv = lift_gmv * est_gmv/sd_gmv

In [30]:
nobs_gmv_test = tt_ind_solve_power(effect_size=eff_size_gmv, power=power, alpha=alpha, ratio=ratio)

In [31]:
# ratio = nobs_control/nobs_test
nobs_gmv_control = nobs_gmv_test*ratio

In [32]:
# всего нужно наблюдений, наблюдений в тестовой группе, в контрольной
print(f"""
Всего необходимо наблюдений: {round(nobs_gmv_control + nobs_gmv_test, 1):,}
Наблюдений в тестовой группе: {round(nobs_gmv_test, 1):,}
Наблюдений в контрольной группе: {round(nobs_gmv_control, 1):,}""")


Всего необходимо наблюдений: 2,212,449.4
Наблюдений в тестовой группе: 1,106,224.7
Наблюдений в контрольной группе: 1,106,224.7
